In [1]:
# see https://github.com/mozilla/mozanalysis/issues/40
%set_env OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES

env: OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col, count, udf, count_distinct
import pyspark.sql.functions as F
import pandas as pd
import pyspark.pandas as ps
import numpy as np
from datetime import datetime
from pyspark.sql.types import *

/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
spark = SparkSession \
        .builder \
        .appName('test-spark2') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/09 10:18:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/09 10:18:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
!wc -l /Applications/spark-3.4.1-bin-hadoop3/work/sparkify_event_data.json

In [4]:
# df = spark.sql('''
#                 SELECT ts, userId, sessionId, page, method, status, level, location, userAgent, gender
#                 FROM json.`/Applications/spark-3.4.1-bin-hadoop3/work/sparkify_event_data.json`
#                 LIMIT 100000
#                ''').repartition(2, 'userId').persist()
df = spark.sql('''
                SELECT ts, userId, sessionId, page, method, status, level, location, userAgent, gender
                FROM json.`data/mini_sparkify_event_data.json`
               ''').repartition(2, 'userId').persist()

In [5]:
df.count()

286500

In [6]:
tmp = df.groupby('userId').agg({'ts': 'count'})
print(tmp.rdd.toDebugString().decode("utf-8"))

(2) MapPartitionsRDD[25] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[24] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  SQLExecutionRDD[23] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[22] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[21] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[20] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[19] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  Exchange hashpartitioning(userId#25, 2), REPARTITION_BY_NUM, [plan_id=15]
+- *(1) Project [ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11]
   +- FileScan json [gender#11,level#15,location#16,method#17,page#18,sessionId#20L,status#22L,ts#23L,userAgent#24,userId#25] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/Users/zma/Documents/DSN

In [7]:
# convert to pandas-on-spark
psdf = df.pandas_api()
print(type(df), type(psdf))

<class 'pyspark.sql.dataframe.DataFrame'> <class 'pyspark.pandas.frame.DataFrame'>


In [148]:
@udf("long")
def add_one_1(x):
    return x + 1

def add_one_2(x):
    return x + 1

@pandas_udf("long")
def add_one_3(x: pd.Series) -> pd.Series:
    return x + 1

In [152]:
df.select(add_one_1('ts')).take(1)

[Row(add_one_1(ts)=1538353668001)]

In [153]:
df.select(add_one_2('ts')).take(1)

TypeError: can only concatenate str (not "int") to str

In [206]:
df.select(add_one_2(col('ts'))).take(1)

[Row((ts + 1)=1538353668001)]

In [155]:
df.select(add_one_3('ts')).take(1)

[Row(add_one_3(ts)=1538353668001)]

In [156]:
psdf['ts'].apply(add_one_1)

TypeError: Invalid argument, not a string or column: 1538353668000 of type <class 'int'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [157]:
psdf['ts'].apply(add_one_2)

0      1538353668001
1      1538353834001
2      1538354117001
3      1538354291001
4      1538354598001
5      1538355027001
6      1538355252001
7      1538355459001
8      1538355710001
9      1538355711001
10     1538355745001
11     1538355807001
12     1538355841001
13     1538355842001
14     1538356000001
15     1538356218001
16     1538356359001
17     1538356598001
18     1538356650001
19     1538356651001
20     1538356678001
21     1538356679001
22     1538356686001
23     1538356782001
24     1538356816001
25     1538356824001
26     1538356889001
27     1538356954001
28     1538357103001
29     1538357104001
30     1538357110001
31     1538357328001
32     1538357377001
33     1538357543001
34     1538357578001
35     1538357746001
36     1538357915001
37     1538357916001
38     1538357996001
39     1538358102001
40     1538358191001
41     1538358192001
42     1538358264001
43     1538358503001
44     1538359745001
45     1538359961001
46     1538360117001
47     153836

In [158]:
psdf['ts'].apply(add_one_3)

TypeError: Type <class 'pandas.core.series.Series'> was not understood.

In [252]:
@udf('float')
def get_hour_1(x):
    return (np.max(x) - np.min(x))/3600/1000

def get_hour_2(x):
    return (F.max(x) - F.min(x))/3600/1000

@pandas_udf('float')
def get_hour_3(x: pd.Series) -> any:
    return (x.max() - x.min())/3600/1000

def get_hour_4(x) -> float:
    return (x.max() - x.min())/3600/1000

def get_hour_5(x) -> pd.DataFrame[str, [float]]:
    return (x.max() - x.min())/3600/1000

In [254]:
spark.udf.register('get_hour_1_', get_hour_1)
spark.udf.register('get_hour_2_', get_hour_2)
spark.udf.register('get_hour_3_', get_hour_3)
spark.udf.register('get_hour_4_', get_hour_4)
spark.udf.register('get_hour_5_', get_hour_5)

23/07/09 08:44:52 WARN SimpleFunctionRegistry: The function get_hour_1_ replaced a previously registered function.
23/07/09 08:44:52 WARN SimpleFunctionRegistry: The function get_hour_2_ replaced a previously registered function.
23/07/09 08:44:52 WARN SimpleFunctionRegistry: The function get_hour_3_ replaced a previously registered function.
23/07/09 08:44:52 WARN SimpleFunctionRegistry: The function get_hour_4_ replaced a previously registered function.


<function __main__.get_hour_5(x) -> Tuple[pyspark.pandas.typedef.typehints.IndexNameType, pyspark.pandas.typedef.typehints.NameType]>

In [168]:
df.select(get_hour_1('ts')).collect()

23/07/08 22:30:37 ERROR Executor: Exception in task 1.0 in stage 884.0 (TID 572)
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:95)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.ha

Py4JJavaError: An error occurred while calling o13958.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 884.0 failed 1 times, most recent failure: Lost task 1.0 in stage 884.0 (TID 572) (192.168.1.5 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonEx

In [173]:
df.select(get_hour_2('ts')).collect()

[Row((((max(ts) - min(ts)) / 3600) / 1000)=1513.1552777777777)]

In [174]:
df.select(get_hour_2(df.ts)).collect()

[Row((((max(ts) - min(ts)) / 3600) / 1000)=1513.1552777777777)]

In [175]:
df.select(get_hour_3('ts')).collect()

[Row(get_hour_3(ts)=1513.1552734375)]

In [244]:
df.select(get_hour_4('ts')).collect()

AttributeError: 'str' object has no attribute 'max'

In [245]:
get_hour_4(psdf['ts'])

1513.1552777777777

In [249]:
psdf['ts'].apply(get_hour_4)

23/07/09 08:41:55 ERROR Executor: Exception in task 0.0 in stage 1055.0 (TID 666)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/accessors.py", line 955, in pudf
    return first_series(output_func(pandas_concat(*series)))
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/groupby.py", line 2279, in rename_output
    pdf = func(pdf)
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/accessors.py", line 946, in apply_func
    return func(first_series(pdf)).to_frame()
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/series.py", line 3772, in apply_each
    return s.apply(func, args=args, **kwds)
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pandas/core/series.py", line 4630, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/opt/anaconda

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/accessors.py", line 955, in pudf
    return first_series(output_func(pandas_concat(*series)))
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/groupby.py", line 2279, in rename_output
    pdf = func(pdf)
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/accessors.py", line 946, in apply_func
    return func(first_series(pdf)).to_frame()
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/pandas/series.py", line 3772, in apply_each
    return s.apply(func, args=args, **kwds)
  File "/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pandas/core/series.py", line 4630, in apply
    return SeriesApply(self, func, convert_dtype, args, kwargs).apply()
  File "/opt/anaconda3/envs/pyspark/lib/python

In [248]:
psdf[['ts']].apply(get_hour_4)

0      232.885278
1      164.629444
2      145.291667
3      173.256944
4      161.263611
5      199.976667
6      164.378611
7      155.348056
8     1457.794722
9     1504.033611
10     622.928056
11     602.393056
12     112.420278
13     105.283611
14     130.655833
15      96.646111
16     125.369167
17     124.300833
18      82.996389
19     114.896111
20     103.280833
21     114.151944
22     120.469167
23     113.562222
24    1457.269444
25    1418.597500
26     776.319444
27    1450.814722
28     823.129722
dtype: float64

In [212]:
df.groupby('userId').agg(get_hour_1('ts'))

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "ts" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(ts)" if you do not care which of the values within a group is returned.;
Aggregate [userId#25], [userId#25, get_hour_1(ts#23L)#66010 AS get_hour_1(ts)#66011]
+- RepartitionByExpression [userId#25], 2
   +- Project [ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11]
      +- Relation [artist#8,auth#9,firstName#10,gender#11,itemInSession#12L,lastName#13,length#14,level#15,location#16,method#17,page#18,registration#19L,sessionId#20L,song#21,status#22L,ts#23L,userAgent#24,userId#25] json


In [214]:
df.groupby('userId').agg(get_hour_2('ts')).collect()

[Row(userId='74', (((max(ts) - min(ts)) / 3600) / 1000)=1434.793888888889),
 Row(userId='', (((max(ts) - min(ts)) / 3600) / 1000)=1462.9155555555556),
 Row(userId='4', (((max(ts) - min(ts)) / 3600) / 1000)=1455.2494444444444),
 Row(userId='45', (((max(ts) - min(ts)) / 3600) / 1000)=1397.44),
 Row(userId='87', (((max(ts) - min(ts)) / 3600) / 1000)=465.0336111111111),
 Row(userId='131', (((max(ts) - min(ts)) / 3600) / 1000)=1434.368888888889),
 Row(userId='126', (((max(ts) - min(ts)) / 3600) / 1000)=1445.9644444444446),
 Row(userId='146', (((max(ts) - min(ts)) / 3600) / 1000)=1448.823611111111),
 Row(userId='36', (((max(ts) - min(ts)) / 3600) / 1000)=1351.5075),
 Row(userId='120', (((max(ts) - min(ts)) / 3600) / 1000)=1429.6455555555556),
 Row(userId='27', (((max(ts) - min(ts)) / 3600) / 1000)=1308.288888888889),
 Row(userId='140', (((max(ts) - min(ts)) / 3600) / 1000)=1444.5180555555555),
 Row(userId='81', (((max(ts) - min(ts)) / 3600) / 1000)=1447.2408333333333),
 Row(userId='137', (((

In [215]:
df.groupby('userId').agg(get_hour_3('ts')).collect()

[Row(userId='', get_hour_3(ts)=1462.91552734375),
 Row(userId='100002', get_hour_3(ts)=1322.2308349609375),
 Row(userId='100004', get_hour_3(ts)=1367.8416748046875),
 Row(userId='100008', get_hour_3(ts)=1175.987548828125),
 Row(userId='100009', get_hour_3(ts)=621.314697265625),
 Row(userId='100010', get_hour_3(ts)=1061.2271728515625),
 Row(userId='100012', get_hour_3(ts)=698.5610961914062),
 Row(userId='100015', get_hour_3(ts)=1289.88720703125),
 Row(userId='100016', get_hour_3(ts)=1368.6336669921875),
 Row(userId='100017', get_hour_3(ts)=3.3211112022399902),
 Row(userId='100018', get_hour_3(ts)=1287.425537109375),
 Row(userId='100021', get_hour_3(ts)=1090.97412109375),
 Row(userId='100023', get_hour_3(ts)=224.96527099609375),
 Row(userId='102', get_hour_3(ts)=1348.0191650390625),
 Row(userId='104', get_hour_3(ts)=1272.3880615234375),
 Row(userId='106', get_hour_3(ts)=716.5999755859375),
 Row(userId='107', get_hour_3(ts)=1071.6707763671875),
 Row(userId='110', get_hour_3(ts)=1415.38500

In [253]:
psdf.groupby('userId')['ts'].apply(get_hour_5)

,c0
,1462.915556
100002,1322.230833
100004,1367.841667
100008,1175.987500
100009,621.314722
100010,1061.227222
100012,698.561111
100015,1289.887222
100016,1368.633611
100017,3.321111


In [241]:
psdf.groupby('userId')['ts'].apply(get_hour_4)

0      1462.915556
1      1322.230833
2      1367.841667
3      1175.987500
4       621.314722
5      1061.227222
6       698.561111
7      1289.887222
8      1368.633611
9         3.321111
10     1287.425556
11     1090.974167
12      224.965278
13     1348.019167
14     1272.388056
15      716.600000
16     1071.670833
17     1415.385000
18     1337.142222
19     1140.391667
20      460.968611
21     1429.645556
22      353.128333
23      390.231389
24     1445.964444
25     1434.368889
26        0.374167
27     1407.880556
28     1303.633889
29      496.096111
30     1444.518056
31     1448.823611
32     1194.276389
33      345.748056
34     1249.442500
35      761.023056
36      619.868056
37        0.116667
38      256.156944
39     1380.511667
40     1091.923333
41      732.434167
42       66.387778
43     1013.257778
44      651.343889
45      837.647222
46     1308.288889
47     1039.702222
48      650.835833
49     1283.128333
50      699.225278
51     1040.416389
52        7.

In [240]:
psdf.groupby('userId')['ts'].apply(get_hour_3)

TypeError: Type <built-in function any> was not understood.

In [260]:
psdf.columns

Index(['ts', 'userId', 'sessionId', 'page', 'method', 'status', 'level',
       'location', 'userAgent', 'gender', 'ts_plus_1'],
      dtype='object')

In [261]:
psdf.groupby('userId').agg({'ts':'get_hour_4_', 'userAgent': 'max'})

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "ts" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(ts)" if you do not care which of the values within a group is returned.;
Aggregate [__index_level_0__#70705], [__index_level_0__#70705, get_hour_4_(ts#23L)#70732 AS ts#70706, max(userAgent#24) AS userAgent#70707]
+- Project [userId#25 AS __index_level_0__#70705, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, pudf(__index_level_0__#588L, ts#23L)#56760L AS ts_plus_1#56763L]
   +- Project [__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, monotonically_increasing_id() AS __natural_order__#600L]
      +- AttachDistributedSequence[__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, me

In [259]:
psdf.groupby('userId').agg({'ts':'get_hour_3_', 'userAgent': 'max'})

AnalysisException: [INVALID_PANDAS_UDF_PLACEMENT] The group aggregate pandas UDF `get_hour_3_` cannot be invoked together with as other, non-pandas aggregate functions.

AnalysisException: [INVALID_PANDAS_UDF_PLACEMENT] The group aggregate pandas UDF `get_hour_3_` cannot be invoked together with as other, non-pandas aggregate functions.

In [238]:
psdf.groupby('userId').agg({'ts':'get_hour_3_'})

,ts
userId,
,1462.915527
100002,1322.230835
100004,1367.841675
100008,1175.987549
100009,621.314697
100010,1061.227173
100012,698.561096
100015,1289.887207
100016,1368.633667


In [239]:
psdf.groupby('userId').agg({'ts':'get_hour_2_'})

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "ts" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(ts)" if you do not care which of the values within a group is returned.;
Aggregate [__index_level_0__#67789], [__index_level_0__#67789, get_hour_2_(ts#23L)#67815 AS ts#67790]
+- Project [userId#25 AS __index_level_0__#67789, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, pudf(__index_level_0__#588L, ts#23L)#56760L AS ts_plus_1#56763L]
   +- Project [__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, monotonically_increasing_id() AS __natural_order__#600L]
      +- AttachDistributedSequence[__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, locatio

In [219]:
psdf.groupby('userId').agg({'ts':'get_hour_3'})

AnalysisException: [UNRESOLVED_ROUTINE] Cannot resolve function `get_hour_3` on search path [`system`.`builtin`, `system`.`session`, `spark_catalog`.`default`].; line 1 pos 0

In [220]:
psdf.groupby('userId').agg({'ts':'get_hour_4'})

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "ts" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(ts)" if you do not care which of the values within a group is returned.;
Aggregate [__index_level_0__#66538], [__index_level_0__#66538, get_hour_4(ts#23L)#66564 AS ts#66539]
+- Project [userId#25 AS __index_level_0__#66538, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, pudf(__index_level_0__#588L, ts#23L)#56760L AS ts_plus_1#56763L]
   +- Project [__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, monotonically_increasing_id() AS __natural_order__#600L]
      +- AttachDistributedSequence[__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location

In [223]:
psdf.groupby('userId')['ts'].apply(get_hour_4)

0      1462.915556
1      1322.230833
2      1367.841667
3      1175.987500
4       621.314722
5      1061.227222
6       698.561111
7      1289.887222
8      1368.633611
9         3.321111
10     1287.425556
11     1090.974167
12      224.965278
13     1348.019167
14     1272.388056
15      716.600000
16     1071.670833
17     1415.385000
18     1337.142222
19     1140.391667
20      460.968611
21     1429.645556
22      353.128333
23      390.231389
24     1445.964444
25     1434.368889
26        0.374167
27     1407.880556
28     1303.633889
29      496.096111
30     1444.518056
31     1448.823611
32     1194.276389
33      345.748056
34     1249.442500
35      761.023056
36      619.868056
37        0.116667
38      256.156944
39     1380.511667
40     1091.923333
41      732.434167
42       66.387778
43     1013.257778
44      651.343889
45      837.647222
46     1308.288889
47     1039.702222
48      650.835833
49     1283.128333
50      699.225278
51     1040.416389
52        7.

In [231]:
psdf.groupby('userId').agg({'ts':'get_hour_4_'})

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "ts" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(ts)" if you do not care which of the values within a group is returned.;
Aggregate [__index_level_0__#67136], [__index_level_0__#67136, get_hour_4_(ts#23L)#67162 AS ts#67137]
+- Project [userId#25 AS __index_level_0__#67136, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, pudf(__index_level_0__#588L, ts#23L)#56760L AS ts_plus_1#56763L]
   +- Project [__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, monotonically_increasing_id() AS __natural_order__#600L]
      +- AttachDistributedSequence[__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, locatio

In [227]:
psdf.groupby('userId').agg(hour=ps.NamedAgg(column='ts', aggfunc='get_hour_4_'))

AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "ts" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(ts)" if you do not care which of the values within a group is returned.;
Aggregate [__index_level_0__#66801], [__index_level_0__#66801, get_hour_4_(ts#23L)#66827 AS (ts, get_hour_4_)#66802]
+- Project [userId#25 AS __index_level_0__#66801, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, pudf(__index_level_0__#588L, ts#23L)#56760L AS ts_plus_1#56763L]
   +- Project [__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, level#15, location#16, userAgent#24, gender#11, monotonically_increasing_id() AS __natural_order__#600L]
      +- AttachDistributedSequence[__index_level_0__#588L, ts#23L, userId#25, sessionId#20L, page#18, method#17, status#22L, le

### I think I am giving up on pandas_api, too many unexpected results. Using the spark.sql functions seems the way to go.

In [272]:
df.select(count_distinct('userId')).collect()

[Row(count(DISTINCT userId)=226)]

In [274]:
df.groupby('userId').agg(count('*')).collect()

[Row(userId='74', count(1)=2887),
 Row(userId='', count(1)=8346),
 Row(userId='4', count(1)=2442),
 Row(userId='45', count(1)=1776),
 Row(userId='87', count(1)=1003),
 Row(userId='131', count(1)=1863),
 Row(userId='126', count(1)=3102),
 Row(userId='146', count(1)=837),
 Row(userId='36', count(1)=1399),
 Row(userId='120', count(1)=1842),
 Row(userId='27', count(1)=291),
 Row(userId='140', count(1)=6880),
 Row(userId='81', count(1)=2335),
 Row(userId='137', count(1)=209),
 Row(userId='65', count(1)=2544),
 Row(userId='29', count(1)=3603),
 Row(userId='70', count(1)=1775),
 Row(userId='61', count(1)=1976),
 Row(userId='136', count(1)=2567),
 Row(userId='80', count(1)=474),
 Row(userId='110', count(1)=235),
 Row(userId='121', count(1)=916),
 Row(userId='85', count(1)=4370),
 Row(userId='96', count(1)=2176),
 Row(userId='20', count(1)=2195),
 Row(userId='3', count(1)=254),
 Row(userId='12', count(1)=1064),
 Row(userId='73', count(1)=457),
 Row(userId='17', count(1)=1102),
 Row(userId='86',

In [269]:
df.groupby(['userId','sessionId']).agg({'*':'count'}).groupby('userId').agg({'count(1)':'mean'}).collect()

[Row(userId='74', avg(count(1))=125.52173913043478),
 Row(userId='', avg(count(1))=5.123388581952118),
 Row(userId='4', avg(count(1))=111.0),
 Row(userId='45', avg(count(1))=111.0),
 Row(userId='87', avg(count(1))=40.12),
 Row(userId='131', avg(count(1))=98.05263157894737),
 Row(userId='126', avg(count(1))=103.4),
 Row(userId='146', avg(count(1))=49.23529411764706),
 Row(userId='36', avg(count(1))=69.95),
 Row(userId='120', avg(count(1))=131.57142857142858),
 Row(userId='27', avg(count(1))=32.333333333333336),
 Row(userId='140', avg(count(1))=96.90140845070422),
 Row(userId='81', avg(count(1))=122.89473684210526),
 Row(userId='137', avg(count(1))=69.66666666666667),
 Row(userId='65', avg(count(1))=110.6086956521739),
 Row(userId='29', avg(count(1))=105.97058823529412),
 Row(userId='70', avg(count(1))=98.61111111111111),
 Row(userId='61', avg(count(1))=104.0),
 Row(userId='136', avg(count(1))=116.68181818181819),
 Row(userId='80', avg(count(1))=39.5),
 Row(userId='110', avg(count(1))=47

In [4]:
data = [
    ("James","CA",np.NaN), ("Julia","",None),
    ("Ram",None,200.0), ("Ramya","NULL",np.NAN)
]
df =spark.createDataFrame(data,["name","state","number"])

In [11]:
df.show()

+-----+-----+------+
| name|state|number|
+-----+-----+------+
|James|   CA|   NaN|
|Julia|     |  null|
|  Ram| null| 200.0|
|Ramya| NULL|   NaN|
+-----+-----+------+



In [7]:
df.select([count(c) for c in df.columns]).show()

+-----------+------------+-------------+
|count(name)|count(state)|count(number)|
+-----------+------------+-------------+
|          4|           3|            3|
+-----------+------------+-------------+



In [14]:
df.select(F.when(col('state').isNull(), 1)).show()

+------------------------------------+
|CASE WHEN (state IS NULL) THEN 1 END|
+------------------------------------+
|                                null|
|                                null|
|                                   1|
|                                null|
+------------------------------------+



In [18]:
df.select(F.isnan('number')).show()

+-------------+
|isnan(number)|
+-------------+
|         true|
|        false|
|        false|
|         true|
+-------------+



In [10]:
df.select([F.count(F.when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+-----+------+
|name|state|number|
+----+-----+------+
|   0|    1|     1|
+----+-----+------+

